In [8]:
"""
This Notebook is designed to re-write the XML files so that
non-existent meshes/*.ply files specified within the XML
file are deleted.
"""

import xml.etree.ElementTree as ET
from dotenv import load_dotenv
import os
from os.path import join, dirname
from blender_test_wraper import splitting_a_line

load_dotenv(join('..', '.env'))
BASE_PATH = os.environ.get('BASE_PATH')  # probably /dev/shm/res_plane/
XML_PATH = 'Bl_xml_files/'

# path containing all the fully-generated signal maps.
BASE_PATH_SIONNA = '/home/yl826/3DPathLoss/nc_raytracing/cm_512_Aug10_7e6_isoTx/'

In [1]:

idx_uuid_sig_map = set([f.split('_')[0] + '_' + f.split('_')[1]
                        for f in os.listdir(BASE_PATH_SIONNA)
                        if os.path.isfile(BASE_PATH_SIONNA + f)])

idx_uuid_fpath = join(BASE_PATH, 'step1.5.txt')
f_ptr = open(idx_uuid_fpath, 'r')
lines = f_ptr.readlines()
f_ptr.close()

idx_uuid_res_plane = set([splitting_a_line(line, uuid_incl='y')[-1] for line in lines])

# set of idx_uuid with XML files that can't produce sig map due to error.
bad_idx_uuid = idx_uuid_sig_map ^ idx_uuid_res_plane
print(bad_idx_uuid[0:10], len(bad_idx_uuid))

/Users/zeyuli/Desktop/Duke/0. Su23_Research/Blender_stuff/res_plane/
['0_ca290c79-7674-471b-85df-d24c59f989e7', '1000_08f52680-8afd-4e0a-b2fc-67e2cd2de31f', '1001_d3bcd342-5264-4225-91e4-8cb216e49479', '1002_040dffe2-8172-4924-8504-ee683dc5a063', '1003_343f05ce-4d42-46e1-bf19-03abd05e194f']


In [9]:
# temp_idx_uuid = '6274_6097b38d-e60c-402f-a516-aad0b488e004'
# temp_xml_file = join(BASE_PATH, XML_PATH, temp_idx_uuid, temp_idx_uuid) + '.xml'
# print(xmlfile)

In [10]:
def check_xml_and_write(idx_uuid):
    xmlfile = join(BASE_PATH, XML_PATH, idx_uuid, idx_uuid) + '.xml'
    tree = ET.parse(xmlfile)
    root = tree.getroot()
    meshes_list = [f for f in os.listdir(join(BASE_PATH, XML_PATH, idx_uuid, 'meshes/'))]

    meshes_problem = []  # stores the indices in the tree where the mesh specified in XML doesn't exist as .ply
    xml_all_shapes = root.findall('shape')  # only iterating through the shapes and not other fields
    idx = 0
    # finds all problematic shape indices
    for item in xml_all_shapes:
        for child in item:
            mesh_name = child.get('value')
            if mesh_name.split('/')[0] != 'meshes':
                raise KeyError(str('mesh_name does not start with "meshes/"' + mesh_name))
            if mesh_name.split('/')[1] not in meshes_list:
                meshes_problem.append(idx)
            break  # breaks since we only check the 'name' attribute in each shape entry
        idx += 1
    if len(meshes_problem) == 0:
        print("Meshes don't have problems")
        return
    ply_start_idx = 0
    for item in root:  # finds the index in tree wrt root where the first shape is
        if item.get('type') == 'ply':
            print(ply_start_idx)
            break
        ply_start_idx += 1

    meshes_problem = sorted(meshes_problem)
    # actually removes the problematic shape entries
    for diff_idx, prob_idx in enumerate(meshes_problem):
        for child in root[prob_idx + ply_start_idx - diff_idx]:
            print(child.get('value'))
        root.remove(root[prob_idx + ply_start_idx - diff_idx])

    tree.write(join(BASE_PATH, XML_PATH, idx_uuid, idx_uuid) + '_new.xml')
    return


for temp_idx_uuid in bad_idx_uuid:
    check_xml_and_write(temp_idx_uuid)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/zeyuli/Desktop/Duke/0. Su23_Research/Blender_stuff/res_plane/Bl_xml_files/6274_6097b38d-e60c-402f-a516-aad0b488e004/6274_6097b38d-e60c-402f-a516-aad0b488e004.xml'